In [1]:
#seeding for comparing experiment in part 2
import torch
import json
SEED = 1111
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda:2')

# SNLI Bert
## Second Tutorial
https://towardsdatascience.com/fine-tuning-pre-trained-transformer-models-for-sentence-entailment-d87caf9ec9db
Check his Github code for complete notebook. I never referred to it. Medium was enough.
BERT in keras-tf: https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b

In [2]:
# define macros
BERT_MODEL = 'uklfr/gottbert-base'#'bert-base-german-cased'
#'HooshvareLab/bert-fa-base-uncased' #'GroNLP/bert-base-dutch-cased'
# 'bert-base-multilingual-cased'
# 'bert-base-uncased'

batch_size = 4
batches_per_epoch = 402

# Prepare data

## load the dataset

In [3]:
# !wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip

In [4]:
import pandas as pd

# custom reader needed to handle quotechars
def read_df_custom(file):
    header = 'doc     unit1_toks      unit2_toks      unit1_txt       unit2_txt       s1_toks s2_toks unit1_sent      unit2_sent      dir     nuc_children    sat_children    genre   u1_discontinuous        u2_discontinuous       u1_issent        u2_issent       u1_length       u2_length       length_ratio    u1_speaker      u2_speaker      same_speaker    u1_func u1_pos  u1_depdir       u2_func u2_pos  u2_depdir       doclen  u1_position      u2_position     percent_distance        distance        lex_overlap_words       lex_overlap_length      unit1_case      unit2_case      label'
    header = header.split()
    df = pd.DataFrame(columns=['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label'])
    file = open(file, 'r')

    rows = []
    count = 0 
    for line in file:
        line = line[:-1].split('\t')
        count+=1
        if count ==1: continue
        row = {}
        for column in ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label']:
            index = header.index(column)
            row[column] = line[index]
        rows.append(row)

    df = pd.concat([df, pd.DataFrame.from_records(rows)])
    return df

# we only need specific columns
# train_df = read_df_custom('../processed/nld.rst.nldt_train_enriched.rels')
# test_df = read_df_custom('../processed/nld.rst.nldt_test_enriched.rels')
# val_df = read_df_custom('../processed/nld.rst.nldt_dev_enriched.rels')
train_df = read_df_custom('../processed/deu.rst.pcc_train_enriched.rels')
test_df = read_df_custom('../processed/deu.rst.pcc_test_enriched.rels')
val_df = read_df_custom('../processed/deu.rst.pcc_dev_enriched.rels')
# train_df = read_df_custom('../processed/fas.rst.prstc_train_enriched.rels')
# test_df = read_df_custom('../processed/fas.rst.prstc_test_enriched.rels')
# val_df = read_df_custom('../processed/fas.rst.prstc_dev_enriched.rels')
# train_df = read_df_custom('../processed/zho.rst.sctb_train_enriched.rels') #.tsv2
# test_df = read_df_custom('../processed/zho.rst.sctb_test_enriched.rels')
# val_df = read_df_custom('../processed/zho.rst.sctb_dev_enriched.rels')

## Clean the data

In [5]:
#dropping any empty values
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# train_df = train_df[:500]
# val_df = val_df[:50]
# test_df = test_df[:50]

## Prepare a dataset handler class

In [6]:

import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from sys import path
path.append('/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/allennlp/data/data_loaders/')
from allennlp.data import allennlp_collate#, DataLoader
# from allennlp.data.data_loaders.simple_data_loader import SimpleDataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer
import pandas as pd

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df):
    self.num_labels = -1
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    self.tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=True) # Using a pre-trained BERT tokenizer to encode sentences
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.train_idx = None
    self.val_idx = None
    self.test_idx = None
    self.init_data()

  def init_data(self):
    self.get_label_mapping()
    self.train_data, self.train_idx = self.load_data(self.train_df)
    self.val_data, self.val_idx = self.load_data(self.val_df)
    self.test_data, self.test_idx = self.load_data(self.test_df)

  def get_label_mapping(self):
    labels = {}
    labels_list = list(set(list(self.train_df['label'].unique()) + list(self.test_df['label'].unique()) + list(self.val_df['label'].unique())))
    for i in range(len(labels_list)):
        labels[labels_list[i]] = i
    self.label_dict = labels# {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    # needed later for classification report object to generate precision and recall on test dataset
    self.rev_label_dict = {self.label_dict[k]:k for k in self.label_dict.keys()} 
  
  def add_directionality(self, premise, hypothesis, dir):
    if dir == "1<2":
        # hypothesis = '< ' + hypothesis + ' {'
        hypothesis = hypothesis + ' right'
    else:
        # premise = '} ' + premise + ' >'
        premise = 'left ' + premise
    return premise, hypothesis


  def load_data2(self, df):
    MAX_LEN = 256 # dont need to enforce this now because snli is a sanitized dataset where sentence lenghts are reasonable. otherwise the beert model doesn't have enough parameters to handle long length sentences
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []
    idx = []
    idx_map = {}
    # self.reach = 'reach'

    premise_list = df['unit1_txt'].to_list()
    hypothesis_list = df['unit2_txt'].to_list()
    label_list = df['label'].to_list()
    dir_list = df['dir'].to_list()
    
    self.num_labels = max(self.num_labels, len(df['label'].unique()))

    count=0
    for (premise, hypothesis, label, dir) in zip(premise_list, hypothesis_list, label_list, dir_list):
      print('old: ', premise, hypothesis)
      premise, hypothesis = self.add_directionality(premise, hypothesis, dir)
      print('new:', premise, hypothesis, '\n')
    

  def load_data(self, df):
    MAX_LEN = 256 # dont need to enforce this now because snli is a sanitized dataset where sentence lenghts are reasonable. otherwise the beert model doesn't have enough parameters to handle long length sentences
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []
    idx = []
    idx_map = {}
    # self.reach = 'reach'

    premise_list = df['unit1_txt'].to_list()
    hypothesis_list = df['unit2_txt'].to_list()
    label_list = df['label'].to_list()
    dir_list = df['dir'].to_list()
    
    self.num_labels = max(self.num_labels, len(df['label'].unique()))

    count=0
    for (premise, hypothesis, label, dir) in zip(premise_list, hypothesis_list, label_list, dir_list):
      premise, hypothesis = self.add_directionality(premise, hypothesis, dir)
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False, max_length=MAX_LEN, truncation=True)
      print(premise)
      print(premise_id)
      # print(self.tokenizer.encode("< "))
      # print(self.tokenizer.encode("< this"))
      # print(self.tokenizer.encode("this ."))
      # print(self.tokenizer.encode("this } "))
      # print(self.tokenizer.encode("fffffff } "))
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False, max_length=MAX_LEN, truncation=True)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])

      idx_map[count] = [premise, hypothesis]
      idx.append(count)
      count+=1
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)

    y = torch.tensor(y)
    idx = torch.tensor(idx)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y, idx)
    return dataset, idx_map

  def get_data_loaders(self, batch_size=4, batches_per_epoch=402, shuffle=True): #1609 samples / 64:25=1600 / 402:4=1608
    train_loader_torch = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size,
    )

    val_loader_torch = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size,
    )

    test_loader_torch = DataLoader(
      self.test_data,
      shuffle=False,#shuffle,
      batch_size=batch_size,
    )
    
    train_loader = LoaderWrapper(train_loader_torch, n_step=batches_per_epoch)
    val_loader = LoaderWrapper(val_loader_torch, n_step=batches_per_epoch)
    test_loader = LoaderWrapper(test_loader_torch, n_step=batches_per_epoch)

    return train_loader, val_loader_torch, test_loader_torch

In [7]:
class LoaderWrapper:
    def __init__(self, loader, n_step):
        self.step = n_step
        self.idx = 0
        self.iter_loader = iter(loader)
        self.loader = loader
    
    def __iter__(self):
        return self

    def __len__(self):
        return self.step

    def __next__(self):
        # if reached number of steps desired, stop
        if self.idx == self.step:
            self.idx = 0
            raise StopIteration
        else:
            self.idx += 1
        # while True
        try:
            return next(self.iter_loader)
        except StopIteration:
            # reinstate iter_loader, then continue
            self.iter_loader = iter(self.loader)
            return next(self.iter_loader)

In [8]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df)

train_loader, val_loader, test_loader = mnli_dataset.get_data_loaders(batch_size=batch_size, batches_per_epoch=batches_per_epoch) #64X250
label_dict = mnli_dataset.label_dict # required by custom func to calculate accuracy, bert model
rev_label_dict = mnli_dataset.rev_label_dict # required by custom func to calculate accuracy

OSError: Can't load tokenizer for 'uklfr/gottbert-base'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'uklfr/gottbert-base' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [ ]:
print(label_dict)

{'solutionhood': 0, 'conjunction': 1, 'preparation': 2, 'summary': 3, 'interpretation': 4, 'result': 5, 'evidence': 6, 'elaboration': 7, 'reason': 8, 'circumstance': 9, 'contrast': 10, 'evaluation-n': 11, 'concession': 12, 'cause': 13, 'condition': 14, 'disjunction': 15, 'antithesis': 16, 'means': 17, 'background': 18, 'evaluation-s': 19, 'joint': 20, 'restatement': 21, 'sequence': 22, 'e-elaboration': 23, 'purpose': 24, 'list': 25}


# Define the model

## load pretrained model

In [ ]:
from transformers import BertForSequenceClassification, AdamW
from torch import optim

model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=len(label_dict)).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.6, mode='max', patience=2, min_lr=5e-7, verbose=True)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [ ]:
from CategoricalAccuracy import CategoricalAccuracy as CA
import numpy as np

ca = CA()

x = torch.tensor(np.array([[[1,0,0], [1,0,0], [1,0,0]]]))
y1 = torch.tensor(np.array([[0], [1], [1]]))
y2 = torch.tensor(np.array([[0], [0], [0]]))

ca(x,y1)
print(ca.get_metric(reset=True))
ca(x,y2)
print(ca.get_metric(reset=True))

0.3333333333333333
1.0


## define evaulation metric

In [ ]:
# to evaluate model for train and test. And also use classification report for testing
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

# helper function to calculate the batch accuracy
def multi_acc(y_pred, y_test, allennlp=False):
  if allennlp==False:
    acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
    return acc

def save_cm(y_true, y_pred, target_names, display_zeros=True):
  confusion_matrix = pd.crosstab(y_true, y_pred)
  print(confusion_matrix)
  target_namesx = confusion_matrix.columns
  target_namesy = confusion_matrix.index.values

  fig, ax = plt.subplots(figsize=(10,10))
  s = sns.heatmap(confusion_matrix, xticklabels=target_namesx, yticklabels=target_namesy, annot=True, fmt = '.5g')

  plt.title('CM predicted v actual values')
  plt.xlabel('Pred')
  plt.ylabel('True')
  plt.tight_layout()
  # plt.show()
  # plt.savefig(image_file+exp+'.png')
  # print(image_file+exp+'.png')

# freeze model weights and measure validation / test 
def evaluate_accuracy(model, optimizer, data_loader, rev_label_dict, label_dict, save_path, is_training=True):
  model.eval()
  total_val_acc  = 0
  total_val_loss = 0
  
  #for classification report
  y_true = []
  y_pred = []
  idx_list = []
  premise_list = []
  hypo_list = []
  idx_map = mnli_dataset.val_idx if is_training else mnli_dataset.test_idx

  with torch.no_grad():
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y, idx) in enumerate(data_loader):      
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      
      # loss, prediction = model(pair_token_ids, 
      #                       token_type_ids=seg_ids, 
      #                       attention_mask=mask_ids, 
      #                       labels=labels).values()
      # acc = multi_acc(prediction, labels)

      ############new code#####################

      outputs = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids)
      # probs = F.softmax(outputs, dim=1)
      # max_idx = torch.max(outputs, 1).indices
      # one_hot = F.one_hot(max_idx, outputs.shape[1])

      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, labels)
      acc = multi_acc(outputs, labels)
      ########################################

      total_val_loss += loss.item()
      total_val_acc  += acc.item()

      # log predictions for classification report
      argmax_predictions = torch.argmax(outputs,dim=1).tolist()
      labels_list = labels.tolist()
      assert(len(labels_list)==len(argmax_predictions))
      for p in argmax_predictions: y_pred.append(rev_label_dict[int(p)])
      for l in labels_list: y_true.append(rev_label_dict[l])
      for i in idx.tolist():
        idx_list.append(i)
        premise_list.append(idx_map[i][0])
        hypo_list.append(idx_map[i][1])

  val_acc  = total_val_acc/len(data_loader)
  val_loss = total_val_loss/len(data_loader)
  cr = classification_report(y_true, y_pred)

  idx_json = {'idx': idx_list, 'gold_label': y_true, 'pred_label': y_pred, 'premise': premise_list, 'hypothesis': hypo_list}
  if not is_training: json.dump(idx_json, open(save_path, 'w', encoding='utf8'), ensure_ascii=False)

  # if not is_training:
  #   save_cm(y_true, y_pred, rev_label_dict)
  
  return val_acc, val_loss, cr, model, optimizer

## define bert custom model

In [ ]:
from transformers import BertModel, AutoTokenizer
import torch.nn as nn
class CustomBERTModel(nn.Module):
    #https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model
    def __init__(self, num_labels):
          super(CustomBERTModel, self).__init__()
          self.num_classes = num_labels+1 # zero indexed classes
          print('ASSIGN:', self.num_classes)
          self.bert = BertModel.from_pretrained(BERT_MODEL)
          ### New layers:
          self.linear1 = nn.Linear(768, 512)
          self.linear2 = nn.Linear(512, 256)
          self.linear3 = nn.Linear(256, 128)
          self.linear4 = nn.Linear(128, self.num_classes)
          self.act1 = nn.ReLU() # can i use the same activation object everywhere?
          self.act2 = nn.ReLU()
          self.act3 = nn.ReLU()
          self.drop = nn.Dropout(0.1) 

    def forward(self, pair_token_ids, token_type_ids, attention_mask):
        sequence_output, pooled_output = self.bert(input_ids=pair_token_ids, 
                        token_type_ids=token_type_ids, 
                        attention_mask=attention_mask).values()

        # sequence_output has the following shape: (batch_size, sequence_length, 768)
        linear1_output = self.linear1(sequence_output[:,0,:].view(-1,768)) ## extract the 1st token's embeddings
        linear1_output = self.act1(linear1_output)
        linear2_output = self.linear2(linear1_output)
        linear2_output = self.act2(linear2_output)
        linear3_output = self.linear3(linear2_output)
        linear3_output = self.act3(linear3_output)
        linear4_output = self.linear4(linear3_output)
        drop_output = self.drop(linear4_output)
        return drop_output# loss, outputs

# tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
model = CustomBERTModel(mnli_dataset.num_labels) # You can pass the parameters if required to have more flexible model
model.to(device) ## can be gpu
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

ASSIGN: 27


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## define training regime

In [ ]:
### MODIFIED
import time
import traceback
import torch.nn.functional as F
from typing import Optional, Iterable, Dict, Any
from EarlyStopperUtil import MetricTracker


from sklearn.metrics import classification_report

EPOCHS = 100

def train(model, train_loader, val_loader, optimizer, scheduler, rev_label_dict):  
  EarlyStopper = MetricTracker(patience=12, metric_name='+accuracy')

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0

    # logging for scheduler
    losses = []
    accuracies= []

    train_size = 0

    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y, idx) in enumerate(train_loader):
      train_size+=1
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      ############new code#####################

      outputs = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids)
      # outputs = F.log_softmax(outputs, dim=1) # log prob
      # outputs = np.argmax(prob, axis=1) # preds
      # https://stackoverflow.com/questions/43672047/convert-probability-vector-into-target-vector-in-python
      # https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, labels)
      loss.backward()
      acc = multi_acc(outputs, labels)
      optimizer.step()
      ################old code#################

      # loss, prediction = model(pair_token_ids, 
      #                       token_type_ids=seg_ids, 
      #                       attention_mask=mask_ids, 
      #                       labels=labels).values()

      # acc = multi_acc(prediction, labels)
      # loss.backward()
      # optimizer.step()

      ########################################
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

      # log losses for scheduler
      losses.append(loss)
      accuracies.append(acc)
      mean_loss = sum(losses)/len(losses)
      scheduler.step(mean_loss)


    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)

    val_acc, val_loss, cr, model, optimizer = evaluate_accuracy(model, optimizer, val_loader, rev_label_dict, label_dict, None)
    EarlyStopper.add_metric(val_acc)
    if EarlyStopper.should_stop_early(): break

    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    print(f'train_size: {train_size}')


In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    train(model, train_loader, val_loader, optimizer, scheduler, rev_label_dict)

Epoch 00007: reducing learning rate of group 0 to 1.2000e-05.
Epoch 00010: reducing learning rate of group 0 to 7.2000e-06.
Epoch 00013: reducing learning rate of group 0 to 4.3200e-06.
Epoch 00016: reducing learning rate of group 0 to 2.5920e-06.
Epoch 00019: reducing learning rate of group 0 to 1.5552e-06.
Epoch 00022: reducing learning rate of group 0 to 9.3312e-07.
Epoch 00025: reducing learning rate of group 0 to 5.5987e-07.
Epoch 00028: reducing learning rate of group 0 to 5.0000e-07.
Epoch 1: train_loss: 2.9150 train_acc: 0.1231 | val_loss: 2.6571 val_acc: 0.1434
00:00:24.54
train_size: 402
Epoch 2: train_loss: 2.7213 train_acc: 0.1443 | val_loss: 2.5681 val_acc: 0.2254
00:00:24.94
train_size: 402
Epoch 3: train_loss: 2.6162 train_acc: 0.1872 | val_loss: 2.4666 val_acc: 0.2336
00:00:24.87
train_size: 402
Epoch 4: train_loss: 2.5123 train_acc: 0.2226 | val_loss: 2.4659 val_acc: 0.2213
00:00:24.59
train_size: 402
Epoch 5: train_loss: 2.3902 train_acc: 0.2500 | val_loss: 2.3357 val

In [ ]:
# save_path = 'bert-disrpt_fas_'+str(batch_size)+'X'+str(batches_per_epoch)+'_refactor_1.pt'
save_path = 'bert-disrpt_deu_'+str(batch_size)+'X'+str(batches_per_epoch)+'_refactor.pt'
torch.save(model.state_dict(), save_path)

# test

In [ ]:
def validate(model, test_loader, optimizer, rev_label_dict, label_dict, save_path):
  start = time.time()
  test_acc, test_loss, cr, model, optimizer = evaluate_accuracy(model, optimizer, test_loader, rev_label_dict, label_dict, save_path.replace('.pt', '.json'), is_training=False)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)

  print(f'Test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  print(cr)

  return test_loss, test_acc


model.load_state_dict(torch.load(save_path))
test_loss, test_acc = validate(model, test_loader, optimizer, rev_label_dict, label_dict, save_path)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test_loss: 5.0918 test_acc: 0.2385
00:00:00.74
                precision    recall  f1-score   support

    antithesis       0.20      0.06      0.09        18
    background       0.29      0.35      0.32        17
         cause       0.12      0.50      0.20         2
  circumstance       0.00      0.00      0.00        15
    concession       0.67      0.31      0.42        13
     condition       0.47      0.78      0.58         9
   conjunction       0.33      0.29      0.31         7
      contrast       0.00      0.00      0.00         8
 e-elaboration       1.00      0.18      0.31        11
   elaboration       0.14      0.50      0.22        10
  evaluation-n       0.00      0.00      0.00         3
  evaluation-s       0.20      0.06      0.09        17
      evidence       0.33      0.20      0.25        10
interpretation       0.03      0.08      0.04        12
         joint       0.27      0.48      0.35        29
          list       0.47      0.27      0.34        26


/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [ ]:
45.10+41.89+40.20

127.19000000000001